In [3]:
import numpy as np
import glob

In [2]:
# -----------------------------------------------------------------------------
# vis_calibrate
# -----------------------------------------------------------------------------
def vis_calibrate(data):
    """Calibrate visible channels to reflectance."""
    solar_irradiance = np.array(2017)
    esd = np.array(0.99)
    factor = np.pi * esd * esd / solar_irradiance

    return data * np.float32(factor) * 100


# -----------------------------------------------------------------------------
# ir_calibrate
# -----------------------------------------------------------------------------
def ir_calibrate(data):
    """Calibrate IR channels to BT."""
    fk1 = np.array(13432.1),
    fk2 = np.array(1497.61),
    bc1 = np.array(0.09102),
    bc2 = np.array(0.99971),

    # if self.clip_negative_radiances:
    #     min_rad = self._get_minimum_radiance(data)
    #     data = data.clip(min=data.dtype.type(min_rad))

    res = (fk2 / np.log(fk1 / data + 1) - bc1) / bc2
    return res


# -----------------------------------------------------------------------------
# ConvertABIToReflectanceBT
# -----------------------------------------------------------------------------
class ConvertABIToReflectanceBT(object):
    """
    Performs scaling of MODIS TOA data
    - Scales reflectance percentages to reflectance units (% -> (0,1))
    - Performs per-channel minmax scaling for emissive bands (k -> (0,1))
    """

    def __init__(self):

        self.reflectance_indices = [0, 1, 2, 3, 4, 6]
        self.emissive_indices = [5, 7, 8, 9, 10, 11, 12, 13]

    def __call__(self, img):

        # Reflectance % to reflectance units
        img[:, :, self.reflectance_indices] = \
            vis_calibrate(img[:, :, self.reflectance_indices])

        # Brightness temp scaled to (0,1) range
        img[:, :, self.emissive_indices] = ir_calibrate(
            img[:, :, self.emissive_indices])

        return img

In [6]:
translation = [1, 2, 0, 4, 5, 6, 3, 8, 9, 10, 11, 13, 14, 15]
chippath = "/explore/nobackup/projects/pix4dcloud/szhang16/abiChips/GOES-16"

In [7]:
chippaths = glob.glob(chippath + "/*.npz")

In [16]:
channel_mins = np.zeros(14, dtype="float") + 100000
channel_maxs = np.zeros(14, dtype="float")
conversion = ConvertABIToReflectanceBT()

In [23]:
for path in chippaths:
    chip = np.load(path)["chip"]
    chip = chip[..., translation]
    chip = conversion.__call__(chip)
    for i in range(14):
        channel_mins[i] = np.minimum(channel_mins[i], np.min(chip[:, :, i]))
        channel_maxs[i] = np.maximum(channel_maxs[i], np.max(chip[:, :, i]))
    

In [24]:
for i in range(14):
    channeltype = "reflectance" if i < 5 or i == 6 else "emissive"
    print(f'Channel {i+1} {channeltype}: {channel_mins[i]} - {channel_maxs[i]}')

Channel 1 reflectance: 0.025723284110426903 - 96.01878356933594
Channel 2 reflectance: -1.2622402906417847 - 56.966243743896484
Channel 3 reflectance: 4.098858833312988 - 122.7410659790039
Channel 4 reflectance: -0.17457956075668335 - 14.47913646697998
Channel 5 reflectance: -0.022678880020976067 - 4.54754638671875
Channel 6 emissive: 117.04326629638672 - 228.68850708007812
Channel 7 reflectance: -0.2152758687734604 - 12.093914031982422
Channel 8 emissive: 152.00592041015625 - 224.83987426757812
Channel 9 emissive: 157.96591186523438 - 242.3325958251953
Channel 10 emissive: 176.1534881591797 - 307.4200439453125
Channel 11 emissive: 210.6049346923828 - 290.8879089355469
Channel 12 emissive: 209.71241760253906 - 344.21173095703125
Channel 13 emissive: 218.10147094726562 - 345.72894287109375
Channel 14 emissive: 225.98939514160156 - 323.5238952636719


In [25]:
print(channel_mins)

[ 2.57232841e-02 -1.26224029e+00  4.09885883e+00 -1.74579561e-01
 -2.26788800e-02  1.17043266e+02 -2.15275869e-01  1.52005920e+02
  1.57965912e+02  1.76153488e+02  2.10604935e+02  2.09712418e+02
  2.18101471e+02  2.25989395e+02]
